# import声明

In [1]:
import pandas as pd
import numpy as np
# import faiss
from tqdm import tqdm
import time

#  1.读取train数据

In [2]:
print('start')

start = time.time()
train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
###处理 behaviors.tsv 行为数据
train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
impressionLog = train_behavior['clickHistory'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)

train_data = pd.DataFrame()
train_data= train_behavior[['id']].join(pd.DataFrame(impressionLog,columns=['clickHistory']))

train_data.dropna(axis=0, how='any', inplace=True)
print(train_data.shape[0])
print('time is : ',time.time() - start)


start
5107639
time is :  29.72873568534851


# 2.读取test数据

In [3]:
print('start')

start = time.time()
test_behavior = pd.read_csv('./data/test/behaviors.tsv',sep ='\t',header=None)
###处理 behaviors.tsv 行为数据
test_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
impressionLog = test_behavior['clickHistory'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)

test_data = pd.DataFrame()
test_data= test_behavior[['id']].join(pd.DataFrame(impressionLog,columns=['clickHistory']))

test_data.dropna(axis=0, how='any', inplace=True)
print(test_data.shape[0])
print('time is : ',time.time() - start)

start
2362514
time is :  9.256201982498169


# 3.合并train和test

In [4]:
all_data = pd.concat([train_data,test_data],axis=0).reset_index().drop('index',1)


# 4.读取用户ID和历史点击新闻ID

In [5]:

## 用户id映射
id_numbers = all_data['id'].drop_duplicates().shape[0]
pre_id = all_data['id'].drop_duplicates().tolist()
now_id = {}
for index in tqdm(range(id_numbers)):
    now_id[pre_id[index]] = index+1
curr_id = all_data['id'].tolist()
for index in tqdm(range(len(curr_id))):
    curr_id[index] = now_id[curr_id[index]]
    
## 历史点击新闻ID映射    
his_numbers = all_data['clickHistory'].drop_duplicates().shape[0]
pre_his = all_data['clickHistory'].drop_duplicates().tolist()
now_his = {}
for index in tqdm(range(his_numbers)):
    now_his[pre_his[index]] = id_numbers+index+1
curr_his = all_data['clickHistory'].tolist()
for index in tqdm(range(len(curr_id))):
    curr_his[index] = now_his[curr_his[index]] 

## 用户id反向映射
index_user = {}
index_user[0] = 'None'
for index in tqdm(range(id_numbers)):
    index_user[index+1] = pre_id[index]
    
##新闻id反向映射
index_news = {}
index_news[0] = 'None'
for index in tqdm(range(his_numbers)):
    index_news[id_numbers+index+1] = pre_his[index]
    
##数据存储
data = pd.DataFrame()
data['id'] = curr_id
data['his'] = curr_his
data.to_csv('./data/id_news.csv',index=0,header=0,sep='\t')

100%|██████████| 44908/44908 [00:00<00:00, 842138.74it/s]


# 5. 通过node2vec得到用户embedding


In [3]:
!pecanpy --input ./data/id_news.csv  --output ./data/news1.emb --mode SparseOTF --workers 44 --verbose --walk-length 100 



Took 0:0:25.74 to load graph
Took 0:0:0.00 to pre-compute transition probabilities
Took 0:5:13.74 to generate walks
Took 0:5:55.29 to train embeddings


# 6.用户ID获取

In [6]:

news_emb = pd.read_csv('./data/news1.emb',header=None)
data = news_emb[0].str.split(' ',expand=True)
new_data = pd.DataFrame()
new_data['user'] = data[0]
new_data['emb'] = data[[x for x in range(1,129)]].values.tolist()
new_data = new_data[new_data['user'].astype(int)<91936]
new_data['user'] = new_data['user'].astype(int)
new_data.sort_values('user',inplace=True)
new_data.to_csv('./data/user_emb1.csv',index=False)

# 7.计算用户相似度

# 8.相似用户存储

In [8]:
relation = pd.concat([neighbor_user,neighbor_news],0)
relation_dict = dict()
for index,row in tqdm(relation.iterrows()):
    relation_dict[row['user']] = row['id']

136843it [00:20, 6833.04it/s]


In [9]:
data = pd.DataFrame()
news_emb = pd.read_csv('./data/news1.emb',header=None)
temp = news_emb[0].str.split(' ',expand=True)

data['user'] = temp[0]
data['emb'] = temp[[x for x in range(1,129)]].values.tolist()
data['user'] = data['user'].apply(lambda x : relation_dict[int(x)])



In [10]:
data.to_csv('./data/users_emb1.csv',index=False)

# 9.user-item排序

In [11]:
new_data = pd.read_csv('./data/users_emb1.csv')

In [12]:
##获取user matrix 
temp_user = {}
temp_news = {}
for index,row in tqdm(new_data.iterrows()):
    user = row['user']
    if user.startswith('U'):
        temp_user[user] = row['emb']
    else:
        temp_news[user] = row['emb']
user_matrix = pd.DataFrame()
user_matrix['key'] = list(temp_user.keys())
user_matrix['value'] = list(temp_user.values())
user_matrix['value'] = user_matrix['value'].apply(lambda x : eval(x))
user_matrix['value'] = user_matrix['value'].apply(lambda x : [float(y) for y in x])
user_matrix.to_csv('./data/user_matrix1.csv',index=False)
#获取item matrix
news_matrix = pd.DataFrame()
news_matrix['key'] = list(temp_news.keys())
news_matrix['value'] = list(temp_news.values())
news_matrix['value'] = news_matrix['value'].apply(lambda x : eval(x))
news_matrix['value'] = news_matrix['value'].apply(lambda x : [float(y) for y in x])
news_matrix.to_csv('./data/news_matrix1.csv',index=False)

136844it [00:21, 6383.07it/s]


In [13]:
##计算user-item 点积
temp = np.matmul(np.array(user_matrix['value'].values.tolist()),np.array(news_matrix['value'].values.tolist()).T) 
for index in tqdm(range(temp.shape[0])):
    q = temp[index]
    temp[index] =  q / q.sum()
data = pd.DataFrame(temp,columns=list(temp_news.keys()))
data['user'] = list(temp_user.keys())

100%|██████████| 91935/91935 [00:22<00:00, 4070.11it/s]


In [14]:
data

,N10885,N33552,N1965,N36310,N44312,N25654,N6188,N42600,N21281,N61666,...,N5633,N21563,N42361,N1030,N55315,N54374,N18718,N34667,N58686,user
0,0.000005,0.000011,0.000011,0.000006,0.000008,0.000015,0.000010,0.000008,0.000014,0.000018,...,0.000024,0.000022,0.000018,0.000020,0.000024,0.000013,0.000018,0.000019,0.000021,U63482
1,0.000020,0.000016,0.000006,0.000013,0.000013,0.000017,0.000012,0.000015,0.000011,0.000022,...,0.000024,0.000020,0.000014,0.000022,0.000019,0.000021,0.000015,0.000020,0.000017,U59594
2,0.000015,0.000009,0.000015,0.000014,0.000011,0.000017,0.000015,0.000013,0.000008,0.000015,...,0.000021,0.000034,0.000017,0.000020,0.000019,0.000012,0.000017,0.000024,0.000024,U44701
3,0.000023,0.000016,0.000018,0.000023,0.000020,0.000015,0.000021,0.000024,0.000018,0.000024,...,0.000018,0.000023,0.000016,0.000017,0.000017,0.000020,0.000019,0.000029,0.000022,U88718
4,0.000014,0.000015,0.000013,0.000016,0.000015,0.000025,0.000013,0.000024,0.000015,0.000017,...,0.000019,0.000018,0.000012,0.000021,0.000018,0.000017,0.000017,0.000022,0.000020,U84756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91930,0.000021,0.000015,0.000018,0.000019,0.000020,0.000017,0.000019,0.000017,0.000018,0.000018,...,0.000019,0.000020,0.000021,0.000020,0.000019,0.000017,0.000022,0.000019,0.000020,U7024
91931,0.000021,0.000018,0.000019,0.000019,0.000017,0.000022,0.000019,0.000018,0.000020,0.000018,...,0.000022,0.000022,0.000021,0.000021,0.000020,0.000020,0.000019,0.000020,0.000019,U33599
91932,0.000018,0.000018,0.000015,0.000017,0.000016,0.000016,0.000017,0.000023,0.000013,0.000021,...,0.000018,0.000021,0.000017,0.000018,0.000018,0.000017,0.000021,0.000025,0.000021,U23577
91933,0.000021,0.000023,0.000025,0.000022,0.000024,0.000021,0.000024,0.000020,0.000022,0.000025,...,0.000021,0.000018,0.000021,0.000024,0.000020,0.000022,0.000020,0.000018,0.000018,U69014


In [15]:
##获取前k个item
item_dict = dict()
columns = list(temp_news.keys())
for index in tqdm(range(len(columns))):
    item_dict[index] = columns[index]

user_item = []
columns=list(temp_news.keys())
k = 30
for index in tqdm(range(temp.shape[0])):
    item = temp[index]
    top_index = np.argsort(-item)
    top_temp = []
    
    for index in range(len(top_index[:k])):
        top_temp.append(item_dict[top_index[index]])
    user_item.append(top_temp)

100%|██████████| 91935/91935 [07:23<00:00, 207.20it/s]


In [16]:
##存储user_item前k
data = pd.DataFrame()
data['user'] = list(temp_user.keys())
data['item'] = user_item
data.to_csv('./data/user_item_nearst1.csv',index=False)

In [17]:
data

,user,item
0,U63482,"[N35868, N1717, N27990, N28581, N63637, N20464..."
1,U59594,"[N46619, N63260, N48110, N33724, N25763, N3083..."
2,U44701,"[N6069, N53927, N62671, N41566, N18533, N12520..."
3,U88718,"[N7170, N29141, N34821, N12443, N11036, N40361..."
4,U84756,"[N26459, N46414, N1452, N468, N60629, N53127, ..."
...,...,...
91930,U7024,"[N40191, N33080, N28681, N62671, N38919, N5361..."
91931,U33599,"[N40191, N28681, N62671, N33080, N4519, N38936..."
91932,U23577,"[N62671, N33080, N40191, N28681, N36507, N326,..."
91933,U69014,"[N40191, N33080, N28681, N62671, N24594, N5758..."


In [18]:
#读取user_item前k矩阵
user_item = pd.read_csv('./data/user_item_nearst1.csv')
user_item['item'] = user_item['item'].apply(lambda x : eval(x))
user_item

item_dict = dict()
for index,row in tqdm(user_item.iterrows()):
   item_dict[row['user']] = row['item']

91935it [00:14, 6297.37it/s]


In [19]:
#train补充邻居信息
train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
temp_his = []
for index,row in tqdm(train_behavior.iterrows()):
    if type(row['clickHistory']) == float:
        curr = None
    else:
        curr = row['clickHistory'].split(' ')
        length = len(curr)
        if(length < 15):
            nearst_item = item_dict[row['id']]
            curr.extend(nearst_item[:15-length])
    temp_his.append(curr)
train_behavior['clickHistory'] = temp_his
train_behavior.to_csv('./data/train/trainBehavior_social1.csv',index=False)

156965it [00:32, 4885.41it/s]


In [20]:
#test补充邻居信息
test_behavior = pd.read_csv('./data/test/behaviors.tsv',sep ='\t',header=None)
test_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
temp_his = []
for index,row in tqdm(test_behavior.iterrows()):
    if type(row['clickHistory']) == float:
        curr = None
    else:
        curr = row['clickHistory'].split(' ')
        length = len(curr)
        if(length < 15):
            nearst_item = item_dict[row['id']]
            curr.extend(nearst_item[:15-length])
    temp_his.append(curr)
test_behavior['clickHistory'] = temp_his
test_behavior.to_csv('./data/test/testBehavior_social1.csv',index=False)

73152it [00:12, 5877.06it/s]
